In [1]:
%load_ext autoreload
%autoreload 2

# Import libraries

In [110]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dartwork_mpl as dm

from scipy.stats import gaussian_kde
from matplotlib.ticker import MultipleLocator

In [3]:
dm.use_dmpl_style()

plt.rcParams['font.size'] = 9

# Figure 1.Grids

In [53]:
def make_points(n):
    """
    Point generator.
    """
    if n == 1:
        return np.array([0]), np.array([0])
    
    x = np.linspace(-2 * (n - 1), 2 * (n - 1), n)
    y = x

    x, y = np.meshgrid(x, y)

    return x, y

In [113]:
fig = plt.figure(figsize=(dm.cm2in(17), dm.cm2in(17)))

offset = dm.make_offset(5, -5, fig)

gs = fig.add_gridspec(
    2, 2, left=0.1, right=0.95, bottom=0.1, top=0.95, wspace=0.1, hspace=0.1,
)

axs = gs.subplots(sharex=True, sharey=True)

for n, ax, abc in zip((1, 2, 4, 8), axs.flat, 'abcd'):
    x, y = make_points(n)

    ax.scatter(x, y, s=8, c='dm.red5')
    ax.text(
        0, 1, f'({abc}) {n} $\\times$ {n}', transform=ax.transAxes + offset, weight='normal',
        va='top', ha='left', fontsize=dm.fs(0),
    )

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_xlim(-20, 20)
    ax.set_ylim(-20, 20)
    ax.set_xticks(np.arange(-18, 21, 4))
    ax.set_yticks(np.arange(-18, 21, 4))

    # 1 Minor tick between major ticks.
    ax.xaxis.set_minor_locator(MultipleLocator(2))
    ax.yaxis.set_minor_locator(MultipleLocator(2))
    
for ax in axs[:, 0]:
    ax.set_ylabel('$y$ coordinate [m]')

for ax in axs[1, :]:
    ax.set_xlabel('$x$ coordinate [m]')

dm.save_and_show(fig)

In [114]:
dm.save_formats(fig, './figure/Fig. 1. Grids')

# Figure 2. Parameter set

In [83]:
df = pd.read_csv('./data/fig2/parameter_set.csv')
df['Ground volumetric heat capacity'] /= 1e6
df['Borehole radius'] *= 100
df

,BHE active length,Borehole radius,Ground thermal conductivity,Ground volumetric heat capacity,BHE distance,BHE buried depth
0,99.001465,7.564697,1.980299,2.486816,4.022852,2.137695
1,99.241699,7.564697,1.980299,2.486816,4.022852,2.137695
2,99.001465,7.881104,1.980299,2.486816,4.022852,2.137695
3,99.001465,7.564697,2.077273,2.486816,4.022852,2.137695
4,99.001465,7.564697,1.980299,2.671387,4.022852,2.137695
...,...,...,...,...,...,...
9795,99.537598,7.782959,1.870962,2.643066,3.839258,1.911133
9796,99.537598,7.782959,2.353154,2.142090,3.839258,1.911133
9797,99.537598,7.782959,2.353154,2.643066,4.051367,1.911133
9798,99.537598,7.782959,2.353154,2.643066,3.839258,2.152930


In [8]:
columns = df.columns.to_list()
columns

['BHE active length',
 'Borehole radius',
 'Ground thermal conductivity',
 'Ground volumetric heat capacity',
 'BHE distance',
 'BHE buried depth']

In [124]:
name_dict = {
    'BHE active length' : '$L$ [m]',
    'Borehole radius' : '$r_b$ [cm]',
    'Ground thermal conductivity' : '$k_s$ [W/m $\cdot$ K]',
    'Ground volumetric heat capacity' : '$C_s$ [MJ]', #10^6 으로 나눠서 사용하시면 됩니다. 
    'BHE distance' : '$d$ [m]',
    'BHE buried depth' : '$D$ [m]'
}

# None values are ignored.
range_info = {
    'BHE active length': {
        'lim': (99, 100),
        'ticks': (99.2, 99.8),
        'ticklabels': None,
    },
    'Borehole radius': {
        'lim': (7, 8.5),
        'ticks': (7.2, 8.3),
        'ticklabels': None,
    },
    'Ground thermal conductivity': {
        'lim': (1.5, 2.5),
        'ticks': (1.6, 2.4),
        'ticklabels': None,
    },
    'Ground volumetric heat capacity': {
        'lim': (2, 3),
        'ticks': (2.2, 2.8),
        'ticklabels': None,
    },
    'BHE distance': {
        'lim': (3.8, 4.2),
        'ticks': (3.9, 4.1),
        'ticklabels': None,
    },
    'BHE buried depth': {
        'lim': (1.8, 2.2),
        'ticks': (1.9, 2.1),
        'ticklabels': None,
    },
}


fig = plt.figure(figsize=(dm.cm2in(17), dm.cm2in(17)))

gs = fig.add_gridspec(
    6, 6, left=0.1, right=0.95, bottom=0.1, top=0.95, wspace=0.2, hspace=0.2,    
)

axs = gs.subplots()


# Upper triangle is empty.
for r in range(6):
    for c in range(6):
        ax = axs[r, c]
        
        if r < c:
            ax.set_axis_off()

        if r == c:
            ax.yaxis.set_visible(False)


# Set y axis label.
for r, ax in enumerate(axs[:, 0]):
    ax.set_ylabel(name_dict[columns[r]])


# Set x axis label.
for c, ax in enumerate(axs[-1, :]):
    ax.set_xlabel(name_dict[columns[c]])


# Set diagonal spines.
for i in range(6):
    axs[i, i].spines['left'].set_visible(False)
    axs[i, i].spines['top'].set_visible(False)
    axs[i, i].spines['right'].set_visible(False)


# 1D histogram.
for i in range(6):
    axs[i, i].hist(df[columns[i]], bins=15, color='dm.red5', density=True, alpha=0.5)

# 2D histogram.
for r in range(6):
    for c in range(6):
        ax = axs[r, c]
        
        if r <= c:
            continue

        h, xedges, yedges = np.histogram2d(df[columns[c]], df[columns[r]], bins=15, density=True)

        X, Y = np.meshgrid(xedges, yedges)
        ax.pcolormesh(X, Y, h.T, cmap='Reds', alpha=0.7, vmax=(1.5 * h.max()))


# Set y axis range.
for r in range(6):
    name = columns[r]
    if name not in range_info:
        continue

    for c in range(r):
        ax = axs[r, c]
        # print(r, c)
        if range_info[name]['lim'] is not None:
            # print(name)
            ax.set_ylim(range_info[name]['lim'])

        if range_info[name]['ticks'] is not None:
            ax.set_yticks(range_info[name]['ticks'])

        if range_info[name]['ticklabels'] is not None:
            ax.set_yticklabels(range_info[name]['ticklabels'])

        if c != 0:
            ax.set_yticklabels([])


# Set x axis range.
for c in range(6):
    name = columns[c]
    if name not in range_info:
        continue

    for r in range(6):
        # print(r, c)

        ax = axs[r, c]
        
        if range_info[name]['lim'] is not None:
            # print(name)
            ax.set_xlim(range_info[name]['lim'])

        if range_info[name]['ticks'] is not None:
            ax.set_xticks(range_info[name]['ticks'])

        if range_info[name]['ticklabels'] is not None:
            ax.set_xticklabels(range_info[name]['ticklabels'])

        if r != (6 - 1):
            ax.set_xticklabels([])


# Remove grids.
for ax in axs.flat:
    ax.grid(False)


# Align y axis label.
fig.align_ylabels(axs[:, 0])

dm.save_and_show(fig)

대각선 위치는 y label이 다른데 같이 사용되어 논문용으로는 부적절 하다고 생각됨. KDE와 pair plot을 분리해야 할 것으로 생각됨.

In [125]:
dm.save_formats(fig, './figure/Fig. 2 Parameter sets generated using saltelli sampling')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# Figure 3. Temporal variation of g-function uncertainty

In [11]:
gval_data = {
    i: pd.read_csv(f'./data/fig3/gval_{i}x{i}.csv', index_col=0)
    for i in (1, 2, 4, 8)
}

gval_data[1]

,1.0,2.0,3.0,4.0,5.0,7.0,9.0,11.0,13.0,15.0,...,81915.0,98299.0,114683.0,131067.0,147451.0,163835.0,196603.0,229371.0,262139.0,294907.0
Borefield_1,0.280342,0.522581,0.687601,0.811946,0.911584,1.065918,1.183690,1.278918,1.358851,1.427723,...,5.462624,5.527670,5.580770,5.625269,5.663300,5.696301,5.751012,5.794759,5.830687,5.860794
Borefield_2,0.280342,0.522582,0.687602,0.811947,0.911586,1.065920,1.183692,1.278922,1.358855,1.427728,...,5.463224,5.528327,5.581480,5.626029,5.664105,5.697149,5.751938,5.795756,5.831750,5.861916
Borefield_3,0.255991,0.491003,0.653165,0.775986,0.874677,1.027899,1.145038,1.239859,1.319507,1.388169,...,5.421286,5.486302,5.539377,5.583855,5.621866,5.654851,5.709533,5.753257,5.789165,5.819256
Borefield_4,0.295017,0.541300,0.707899,0.833082,0.933238,1.088177,1.206290,1.301738,1.381822,1.450805,...,5.479905,5.544329,5.596865,5.640846,5.678396,5.710947,5.764832,5.807837,5.843090,5.872578
Borefield_5,0.259015,0.494967,0.657506,0.780530,0.879350,1.032725,1.149955,1.244836,1.324528,1.393224,...,5.436457,5.502409,5.556333,5.601589,5.640323,5.673983,5.729901,5.774737,5.811657,5.842674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Borefield_9796,0.229738,0.456149,0.614864,0.735839,0.833383,0.985258,1.101636,1.195968,1.275278,1.343692,...,5.388272,5.454640,5.508956,5.554585,5.593675,5.627677,5.684239,5.729675,5.767156,5.798699
Borefield_9797,0.365747,0.628840,0.801878,0.930446,1.032686,1.190049,1.309524,1.405838,1.486521,1.555939,...,5.544262,5.604743,5.653722,5.694449,5.728989,5.758738,5.807529,5.845994,5.877164,5.902953
Borefield_9798,0.297152,0.544002,0.710819,0.836116,0.936341,1.091359,1.209515,1.304989,1.385090,1.454085,...,5.471370,5.534801,5.586453,5.629633,5.666450,5.698324,5.750988,5.792914,5.827202,5.855818
Borefield_9799,0.297152,0.544002,0.710819,0.836116,0.936341,1.091359,1.209515,1.304989,1.385090,1.454085,...,5.475670,5.539378,5.591267,5.634653,5.671650,5.703685,5.756627,5.798784,5.833270,5.862058


In [24]:
def make_quntiles(data, n):
    x = data[n].columns.to_numpy().astype(float)
    q = np.quantile(data[n].values, [0.025, 0.5, 0.975], axis=0)

    return x, q


x, q = make_quntiles(gval_data, 1)
q.shape

max_x = x.max()
max_x

294907.0

In [79]:
fig = plt.figure(figsize=(dm.cm2in(17), dm.cm2in(12)))

offset = dm.make_offset(5, -5, fig)

gs = fig.add_gridspec(
    2, 2, left=0.1, right=0.95, bottom=0.1, top=0.90, wspace=0.1, hspace=0.1,
)

axs = gs.subplots(sharex=True, sharey=True)

for n, ax, abc in zip((1, 2, 4, 8), axs.flat, 'abcd'):
    x, y = make_points(n)

    x, q = make_quntiles(gval_data, n)

    line, = ax.plot(x, q[1], c='dm.red5', lw=0.7)
    fill = ax.fill_between(x, q[0], q[2], color='dm.red5', alpha=0.25, lw=0)
    ax.text(
        0, 1, f'({abc}) {n} $\\times$ {n}', transform=ax.transAxes + offset, weight='normal',
        va='top', ha='left', fontsize=dm.fs(0),
    )

    if n == 2:
        ax.legend(
            [line, fill], ['Median', '95% credible interval'],
            loc='lower right', ncols=2, fontsize=dm.fs(-1), bbox_to_anchor=(1.01, 1),
        )

for ax in axs.flat:
    ax.set_xlim(1, max_x)
    ax.set_ylim(0, 60)
    # ax.set_xticks(np.arange(-18, 21, 4))
    # ax.set_yticks(np.arange(-18, 21, 4))
    ax.set_xscale('log')
    
for ax in axs[:, 0]:
    ax.set_ylabel('g-function')

for ax in axs[1, :]:
    ax.set_xlabel('Time [hour]')

dm.save_and_show(fig)

In [80]:
dm.save_formats(fig, './figure/Fig. 3 Temporal variation of g-function uncertainty')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# Figure 4. Transient variation of the first order sensitivity index of parameters impact on g-function

In [98]:
sa_data = {
    i: pd.read_csv(f'./data/fig4_5/SA_{i}x{i}.csv', index_col=0).T
    for i in (1, 2, 4, 8)
}

sa_data[1]

,BHE active length,Borehole radius,Ground thermal conductivity,Ground volumetric heat capacity,BHE distance,BHE buried depth
1.0,-4.717437e-08,0.399644,0.181658,0.431108,0.0,-7.130709e-17
2.0,-7.069054e-08,0.400954,0.182112,0.429781,0.0,-1.102164e-16
3.0,-9.236880e-08,0.401318,0.182217,0.429153,0.0,-3.249285e-16
4.0,-1.123150e-07,0.401508,0.182254,0.428788,0.0,3.363617e-17
5.0,-1.308513e-07,0.401636,0.182268,0.428544,0.0,-3.973285e-16
...,...,...,...,...,...,...
163835.0,1.532315e-04,0.644863,0.110036,0.254179,0.0,1.373954e-03
196603.0,1.992050e-04,0.670893,0.102171,0.235487,0.0,1.576063e-03
229371.0,2.471812e-04,0.694677,0.094964,0.218393,0.0,1.764700e-03
262139.0,2.966019e-04,0.716515,0.088327,0.202687,0.0,1.941593e-03


In [133]:
columns = df.columns.to_list()
columns

['BHE active length',
 'Borehole radius',
 'Ground thermal conductivity',
 'Ground volumetric heat capacity',
 'BHE distance',
 'BHE buried depth']

In [168]:
name_dict = {
    'BHE active length' : '$L$ [m]',
    'Borehole radius' : '$r_b$ [cm]',
    'Ground thermal conductivity' : '$k_s$ [W/m $\cdot$ K]',
    'Ground volumetric heat capacity' : '$C_s$ [MJ]', #10^6 으로 나눠서 사용하시면 됩니다. 
    'BHE distance' : '$d$ [m]',
    'BHE buried depth' : '$D$ [m]'
}

colors = {
    'BHE active length' : 'dm.red5',
    'Borehole radius' : 'dm.blue5',
    'Ground thermal conductivity' : 'dm.green5',
    'Ground volumetric heat capacity' : 'dm.orange5',
    'BHE distance' : 'dm.violet5',
    'BHE buried depth' : 'dm.cyan5'
}

lstyles =  {
    'BHE active length' : '-',
    'Borehole radius' : '--',
    'Ground thermal conductivity' : '-.',
    'Ground volumetric heat capacity' : ':',
    'BHE distance' : (0, (10, 3)),
    'BHE buried depth' : (0, (3, 5, 1, 5, 1, 5)),
}


fig = plt.figure(figsize=(dm.cm2in(17), dm.cm2in(12)))

offset = dm.make_offset(5, -5, fig)

gs = fig.add_gridspec(
    2, 2, left=0.1, right=0.95, bottom=0.1, top=0.90, wspace=0.1, hspace=0.1,
)

axs = gs.subplots(sharex=True, sharey=True)

lines = []
for n, ax, abc in zip((1, 2, 4, 8), axs.flat, 'abcd'):
    for name in columns:
        x = sa_data[n].index.to_numpy().astype(float)
        y = sa_data[n][name].to_numpy()

        line, = ax.plot(x, y, lw=0.7, c=colors[name], ls=lstyles[name])
        lines.append(line)
    
    ax.text(
        0, 1, f'({abc}) {n} $\\times$ {n}', transform=ax.transAxes + offset, weight='normal',
        va='top', ha='left', fontsize=dm.fs(0),
    )

    if n == 1:
        ax.legend(
            lines[:3], [name_dict[name] for name in columns][:3],
            # loc='lower right', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(1.02, 1.02),
            mode='expand', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(0, 1.07, 1, 0.105),
            columnspacing=2.5,
        )

    if n == 2:
        ax.legend(
            lines[3:], [name_dict[name] for name in columns][3:],
            # loc='lower right', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(1.02, 1.02),
            mode='expand', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(0, 1.07, 1, 0.105),
            columnspacing=2.5,
        )

for ax in axs.flat:
    ax.set_xlim(1, max_x)
    ax.set_ylim(-0.03, 1)
    # ax.set_xticks(np.arange(-18, 21, 4))
    # ax.set_yticks(np.arange(-18, 21, 4))
    ax.set_xscale('log')
    
for ax in axs[:, 0]:
    # ax.set_ylabel('First-order sensitivity index')
    ax.set_ylabel('')

fig.text(0.025, 0.5, 'First-order sensitivity index', va='center', rotation='vertical')

for ax in axs[1, :]:
    ax.set_xlabel('Time [hour]')

dm.save_and_show(fig)

In [169]:
dm.save_formats(fig, './figure/Fig. 4 Transient variation of the first order sensitivity index of parameters impact on g-function')

# Figure 5. Transient variation of the first order sensitivity index of influential parameters on g-function

In [170]:
sa_data = {
    i: pd.read_csv(f'./data/fig4_5/SA_{i}x{i}.csv', index_col=0).T
    for i in (1, 2, 4, 8)
}

sa_data[1]

,BHE active length,Borehole radius,Ground thermal conductivity,Ground volumetric heat capacity,BHE distance,BHE buried depth
1.0,-4.717437e-08,0.399644,0.181658,0.431108,0.0,-7.130709e-17
2.0,-7.069054e-08,0.400954,0.182112,0.429781,0.0,-1.102164e-16
3.0,-9.236880e-08,0.401318,0.182217,0.429153,0.0,-3.249285e-16
4.0,-1.123150e-07,0.401508,0.182254,0.428788,0.0,3.363617e-17
5.0,-1.308513e-07,0.401636,0.182268,0.428544,0.0,-3.973285e-16
...,...,...,...,...,...,...
163835.0,1.532315e-04,0.644863,0.110036,0.254179,0.0,1.373954e-03
196603.0,1.992050e-04,0.670893,0.102171,0.235487,0.0,1.576063e-03
229371.0,2.471812e-04,0.694677,0.094964,0.218393,0.0,1.764700e-03
262139.0,2.966019e-04,0.716515,0.088327,0.202687,0.0,1.941593e-03


In [171]:
columns = df.columns.to_list()
columns

['BHE active length',
 'Borehole radius',
 'Ground thermal conductivity',
 'Ground volumetric heat capacity',
 'BHE distance',
 'BHE buried depth']

In [202]:
targets = [
    'Ground volumetric heat capacity',
    'Ground thermal conductivity',
    'Borehole radius',
    'BHE distance',
]

name_dict = {
    'BHE active length' : '$L$ [m]',
    'Borehole radius' : '$r_b$ [cm]',
    'Ground thermal conductivity' : '$k_s$ [W/m $\cdot$ K]',
    'Ground volumetric heat capacity' : '$C_s$ [MJ]', #10^6 으로 나눠서 사용하시면 됩니다. 
    'BHE distance' : '$d$ [m]',
    'BHE buried depth' : '$D$ [m]'
}

colors = {
    1 : 'dm.red2',
    2 : 'dm.red4',
    4 : 'dm.red6',
    8 : 'dm.red8',
}

lstyles =  {
    1 : '-',
    2 : '--',
    4 : '-.',
    8 : ':',
}


fig = plt.figure(figsize=(dm.cm2in(17), dm.cm2in(12)))

offset = dm.make_offset(5, -5, fig)
offset2 = dm.make_offset(20, -6, fig)

gs = fig.add_gridspec(
    2, 2, left=0.1, right=0.95, bottom=0.1, top=0.90, wspace=0.1, hspace=0.1,
)

axs = gs.subplots(sharex=True, sharey=True)

lines = []
for name, ax, abc in zip(targets, axs.flat, 'abcd'):
    for n in (1, 2, 4, 8):
        if name == 'BHE distance' and n == 1:
            continue

        x = sa_data[n].index.to_numpy().astype(float)
        y = sa_data[n][name].to_numpy()

        line, = ax.plot(x, y, lw=0.7, c=colors[n], ls=lstyles[n])
        lines.append(line)
        
    ax.text(
        0, 1, f'({abc})', transform=ax.transAxes + offset, weight='normal',
        va='top', ha='left', fontsize=dm.fs(0),
    )

    ax.text(
        0, 1, f'{name} ({name_dict[name].split()[0]})', transform=ax.transAxes + offset2, weight='normal',
        va='top', ha='left', fontsize=dm.fs(-1),
    )

    if name == targets[1]:
        ax.legend(
            lines, [f'{n} $\\times$ {n}' for n in (1, 2, 4, 8)],
            # loc='lower right', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(1.02, 1.02),
            mode='expand', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(0, 1.07, 1, 0.105),
            columnspacing=2.5,
        )

        # if n == 2:
        #     ax.legend(
        #         lines[3:], [name_dict[name] for name in columns][3:],
        #         # loc='lower right', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(1.02, 1.02),
        #         mode='expand', ncols=6, fontsize=dm.fs(-1), bbox_to_anchor=(0, 1.07, 1, 0.105),
        #         columnspacing=2.5,
        #     )

for ax in axs.flat:
    ax.set_xlim(1, max_x)
    ax.set_ylim(-0.03, 1)
    # ax.set_xticks(np.arange(-18, 21, 4))
    # ax.set_yticks(np.arange(-18, 21, 4))
    ax.set_xscale('log')
    
for ax in axs[:, 0]:
    # ax.set_ylabel('First-order sensitivity index')
    ax.set_ylabel('')

fig.text(0.025, 0.5, 'First-order sensitivity index', va='center', rotation='vertical')

for ax in axs[1, :]:
    ax.set_xlabel('Time [hour]')

dm.save_and_show(fig, size=800)

In [203]:
dm.save_formats(fig, './figure/Fig. 5 Transient variation of the first order sensitivity index of influential parameters on g-function')